### 二次元リストから生成

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

import pandas as pd

# 1 各行を一つのリストとして並べる

list = [
    [1, 100, 0.33, 'AAA', 'AAA100'], 
    [2, 200, 0.67, 'BBB', 'BBB200'], 
    [3, 300, 1, 'CCC', 'CCC300'], 
    [4, 400, 1.33, 'DDD', 'DDD400'], 
    [5, 500, 1.67, 'EEE', 'EEE500'], 
    [6, 600, 2, 'FFF', 'FFF600']
]

# 2 表に変換
# pd.DataFrame(list) : pandasのDataFrameで表に変換
df = pd.DataFrame(list)
# 行名・列名は自動で割り振られた番号になっている

# 3 行名の変更
df.index = ['row0', 'row1', 'row2', 'row3', 'row4', 'row5']

# 4 列名の変更
df.columns = ['col0', 'col1', 'col2', 'col3', 'col4']

df

# 1 各行を一つのリストとして並べる
list = [
[1, 100, 0.33, 'AAA', 'AAA100'], 
[2, 200, 0.67, 'BBB', 'BBB200'], 
[3, 300, 1, 'CCC', 'CCC300'], 
[4, 400, 1.33, 'DDD', 'DDD400'], 
[5, 500, 1.67, 'EEE', 'EEE500'], 
[6, 600, 2, 'FFF', 'FFF600']]


#2 表に変換(オプションで行列名を指定)
ind = ['row0', 'row1', 'row2', 'row3', 'row4', 'row5']
col = ['col0', 'col1', 'col2', 'col3', 'col4']

df = pd.DataFrame(list, index=ind, columns=col)


df

# 1 各行を一つのリストにする
list = [
['row0', 1, 100, 0.33, 'AAA', 'AAA100'], 
['row1', 2, 200, 0.67, 'BBB', 'BBB200'], 
['row2', 3, 300, 1, 'CCC', 'CCC300'], 
['row3', 4, 400, 1.33, 'DDD', 'DDD400'], 
['row4', 5, 500, 1.67, 'EEE', 'EEE500'], 
['row5', 6, 600, 2, 'FFF', 'FFF600'], ]

# 2 表に変換
df = pd.DataFrame(list)

# 3 列名を変更
df.columns = ['', 'col0', 'col1', 'col2', 'col3', 'col4']

# 4 行名となる列を指定
df = df.set_index('')

df

# 1 各行ごとのリストを作成

listA = [1, 100, 0.33, 'AAA', 'AAA100']
listB = [2, 200, 0.67, 'BBB', 'BBB200']
listC = [3, 300, 1, 'CCC', 'CCC300']
listD = [4, 400, 1.33, 'DDD', 'DDD400']
listE = [5, 500, 1.67, 'EEE', 'EEE500']
listF = [6, 600, 2, 'FFF', 'FFF600']

# 2 まとめて表に変換

df = pd.DataFrame([listA, listB, listC, listD, listE, listF])

# 3 行名の変更
df.index = ['row0', 'row1', 'row2', 'row3', 'row4', 'row5']

# 4 列名の変更
df.columns = ['col0', 'col1', 'col2', 'col3', 'col4']

del list

In [ ]:
cg_c_data = return_dict_summary_fixed()

x = cg_c_data["processes"]
y = cg_c_data["ICNVRT"]

cg_icnvrt_lr = ModelLog10(x, y, "CG", "ICNVRT")

cg_icnvrt_lr.calc_lr()

cg_icnvrt_lr.plot_graph()

In [ ]:
max_in_y = max(y)
# 最大値の個数を取得
y.tolist().count(max_in_y)
# 最大値のインデックスで最小のもの
y.tolist().index(max_in_y)

In [ ]:
model_branch = ModelBranch(x, y, "CG", "ICNVRT")

model_branch.calc_lr()

model_branch.calc_mape_score()

print(model_branch.mape_score)

model_branch.plot_graph()

In [ ]:
# ハセガワ手法でMAPE表を作成するための準備

# ipynb形式のライブラリのインポート
%run ./lib.ipynb

import copy
import itertools

# FT についてやってみる
## FTに適した実行プロセス数を備えたリスト
proceses = processes_excludeBTSP
## FTの辞書を作成
## <関数名>:<実行回数のリスト>, rowData:<実行プロセス数のリスト>
FixedClassList_inFT = return_fixed_class(BenchMark="ft", Processes=processes, FixedBenchMarkClass="C")
FixedClassDataFrame_inFT = pd.concat(FixedClassList_inFT, axis=1)
DictData_inFT = return_dict_Data(FixedClassDataFrame_inFT)
## 実行プロセス数のリストから組み合わせのリストを作成
RowData_inFT = copy.deepcopy(DictData_inFT['rowData'])
RowDataExcludeLargestProcess_inFT = RowData_inFT[:-1]
print(f"RowData_inFT={RowData_inFT}")
print(f"RowDataExcludeLargestProcess_inFT={RowDataExcludeLargestProcess_inFT}")
CombinationProcesses_inFT = list(itertools.combinations(RowDataExcludeLargestProcess_inFT, 3))
print(f"CombinationProcesses_inFT={CombinationProcesses_inFT}")

SET_CLASS_inFT = DictData_inFT['SET_CLASS']
SET_CLASSExcludeDataAtLargestProcess_inFT = SET_CLASS_inFT[:-1]
print(f"SET_CLASSExcludeDataAtLargestProcess_inFT={SET_CLASSExcludeDataAtLargestProcess_inFT}")

CombinationSET_CLASSExcludeDataAtLargestProcess_inFT = list(itertools.combinations(SET_CLASSExcludeDataAtLargestProcess_inFT, 3))
print(f"CombinationSET_CLASSExcludeDataAtLargestProcess_inFT={CombinationSET_CLASSExcludeDataAtLargestProcess_inFT}")

for i in range(len(CombinationSET_CLASSExcludeDataAtLargestProcess_inFT)):
    for j in range(len(CombinationSET_CLASSExcludeDataAtLargestProcess_inFT[i])):
        x_index = np.where(RowDataExcludeLargestProcess_inFT == CombinationProcesses_inFT[i][j])
        y_index = np.where(SET_CLASSExcludeDataAtLargestProcess_inFT==CombinationSET_CLASSExcludeDataAtLargestProcess_inFT[i][j])
        if(x_index != y_index):
            print("E", end=", ")
        else:
            print(f"y_index[{y_index}] == x_index[{x_index}]")

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
# 変数：test_ratios
# テスト用データとする割合のリスト
test_ratios = [0.3, 0.4, 0.5, 0.6, 0.7]

##### 実行プロセス数のデータをもとにMAPE表を作成する処理 #####

for test_ratio in test_ratios:
    print(f"test_ratio={test_ratio}")
    print(f"train_list, test_list = {split_by_ratio(processes_excludeBTSP, test_ratio)} on processes_excludeBTSP")
    print(f"train_list, test_list = {split_by_ratio(processes_onlyBTSP, test_ratio)} on processes_onlyBTSP")
    print(f"\n")
    fixed_class_list = [0] * len(benchmarks)
    for i in range(len(fixed_class_list)):
        if ( benchmarks[i] == "bt" or benchmarks[i] == "sp" ):
            processes = processes_onlyBTSP
        else:
            processes = processes_excludeBTSP
        fixed_class_list[i] = return_fixed_class(BenchMark = benchmarks[i], Processes = processes, FixedBenchMarkClass = "C")
    fixed_class_DataFrame = [0] * len(fixed_class_list)
    for i in range(len(fixed_class_list)):
        fixed_class_DataFrame[i] = pd.concat(fixed_class_list[i] , axis=1)

    # テーブルの元となるDataFrameを作成する
    MapeTable_source = [0] * len(fixed_class_list)
    for i in range(len(fixed_class_list)):
        dict_data = return_dict_Data(fixed_class_DataFrame[i])
        MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio)
        MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedClass_{test_ratio}.csv")
        MapeTable_row = return_MapeTable_row(MapeTable_per_benchmark, benchmarks[i])
        MapeTable_source[i] = MapeTable_row
    # テーブルの元となるDataFrameにつける列名のリスト
    MapeTable_source_column_name = ["線形モデル", "対数モデル", "反比例モデル", "分岐モデル", "ベンチマーク名"]
    MapeTable = pd.DataFrame(MapeTable_source)
    MapeTable.columns = MapeTable_source_column_name
    MapeTable = MapeTable.set_index('ベンチマーク名')
    # テーブルを欲しい形でCSVとして出力する
    save_MapeTable(MapeTable, suffix=f"FixedClass{test_ratio}")


In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
BenchmarkClasses = ["A", "B", "C", "D"]
BenchmarkClasses_on_num = [1, 4, 16, 256]
test_ratios = [0.3, 0.5, 0.8]

##### ベンチマーククラスをもとにMAPE表を作成する処理 #####

for test_ratio in test_ratios:
    print(f"test_ratio={test_ratio}")
    print(f"train_list, test_list = {split_by_ratio(BenchmarkClasses, test_ratio)} on BenchmarkClasses")
    print(f"train_list, test_list = {split_by_ratio(BenchmarkClasses_on_num, test_ratio)} on BenchmarkClasses_on_num")
    fixed_process_list = [0] * len(benchmarks)
    for i in range(len(benchmarks)):
        fixed_process_list[i] = return_fixed_process(BenchMark=benchmarks[i], BenchMarkClasses=BenchmarkClasses, FixedProcess=32)

    fixed_process_DataFrame = [0] * len(benchmarks)
    for i in range(len(benchmarks)):
        fixed_process_DataFrame[i] = pd.concat(fixed_process_list[i], axis=1)

    # テーブルの元となるDataFrameを作成する
    MapeTable_source = []
    for i in range(len(fixed_process_list)):
        dict_data = return_dict_Data(fixed_process_DataFrame[i])
        dict_data['rowData'] = BenchmarkClasses_on_num

        try:
            MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio)
        except:
            print(f"MAPEを算出するのに問題が生じました。@{benchmarks[i]}")
            continue
        
        MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio)
        MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedProcess_{test_ratio}.csv")
        MapeTable_row = return_MapeTable_row(MapeTable_per_benchmark, benchmarks[i])
        MapeTable_source.append(MapeTable_row)
    # テーブルの元となるDataFrameにつける列名のリスト
    MapeTable_source_column_name = ["線形モデル", "対数モデル", "反比例モデル", "分岐モデル", "ベンチマーク名"]
    MapeTable = pd.DataFrame(MapeTable_source)
    MapeTable.columns = MapeTable_source_column_name
    MapeTable = MapeTable.set_index('ベンチマーク名')
    # テーブルを欲しい形でCSVとして出力する
    save_MapeTable(MapeTable, suffix=f"FixedProcess{test_ratio}")



In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

##### 訓練データでクラスを固定した際のMAPE表を作成する処理 #####

# 変数：test_ratios
# テスト用データとする割合のリスト
test_ratios = [0.3, 0.4, 0.5, 0.6, 0.7]

for test_ratio in test_ratios:
    print(f"test_ratio={test_ratio}")
    print(f"train_list, test_list = {split_by_ratio(processes_excludeBTSP, test_ratio)} on processes_excludeBTSP")
    print(f"train_list, test_list = {split_by_ratio(processes_onlyBTSP, test_ratio)} on processes_onlyBTSP")
    print(f"\n")
    fixed_class_list = [0] * len(benchmarks)
    for i in range(len(fixed_class_list)):
        if ( benchmarks[i] == "bt" or benchmarks[i] == "sp" ):
            processes = processes_onlyBTSP
        else:
            processes = processes_excludeBTSP
        fixed_class_list[i] = return_fixed_class(BenchMark = benchmarks[i], Processes = processes, FixedBenchMarkClass = "C")
    fixed_class_DataFrame = [0] * len(fixed_class_list)
    for i in range(len(fixed_class_list)):
        fixed_class_DataFrame[i] = pd.concat(fixed_class_list[i] , axis=1)

    # テーブルの元となるDataFrameを作成する
    MapeTable_source = [0] * len(fixed_class_list)
    for i in range(len(fixed_class_list)):
        dict_data = return_dict_Data(fixed_class_DataFrame[i])
        MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio, train=True)
        MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedClassTrain_{test_ratio}.csv")
        MapeTable_row = return_MapeTable_row(MapeTable_per_benchmark, benchmarks[i])
        MapeTable_source[i] = MapeTable_row
    # テーブルの元となるDataFrameにつける列名のリスト
    MapeTable_source_column_name = ["線形モデル", "対数モデル", "反比例モデル", "分岐モデル", "ベンチマーク名"]
    MapeTable = pd.DataFrame(MapeTable_source)
    MapeTable.columns = MapeTable_source_column_name
    MapeTable = MapeTable.set_index('ベンチマーク名')
    # テーブルを欲しい形でCSVとして出力する
    save_MapeTable(MapeTable, suffix=f"FixedClass{test_ratio}")


In [11]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

##### 訓練データでプロセスを固定した際のMAPE表を作成する処理 #####

# 変数：test_ratios
# テスト用データとする割合のリスト
BenchmarkClasses = ["A", "B", "C", "D"]
BenchmarkClasses_on_num = [1, 4, 16, 256]
test_ratios = [0.3, 0.5, 0.8]

for test_ratio in test_ratios:
    print(f"test_ratio={test_ratio}")
    print(f"train_list, test_list = {split_by_ratio(BenchmarkClasses, test_ratio)} on BenchmarkClasses")
    print(f"\n")
    fixed_Process_list = [0] * len(benchmarks)
    for i in range(len(fixed_Process_list)):
        fixed_Process_list[i] = return_fixed_process(BenchMark = benchmarks[i], BenchMarkClasses = BenchmarkClasses, FixedProcess = 128)
    fixed_Process_DataFrame = [0] * len(fixed_Process_list)
    for i in range(len(fixed_Process_list)):
        fixed_Process_DataFrame[i] = pd.concat(fixed_Process_list[i] , axis=1)

    # テーブルの元となるDataFrameを作成する
    MapeTable_source = [0] * len(fixed_Process_list)
    for i in range(len(fixed_Process_list)):
        dict_data = return_dict_Data(fixed_Process_DataFrame[i])
        dict_data['rowData'] = BenchmarkClasses_on_num

        try:
            MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio, train=True)
        except:
            print(f"MAPEを算出するのに問題が生じました。@{benchmarks[i]}")
            continue
        MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedProcessTrain_{test_ratio}.csv")
        MapeTable_row = return_MapeTable_row(MapeTable_per_benchmark, benchmarks[i])
        MapeTable_source[i] = MapeTable_row


test_ratio=0.3
train_list, test_list = (['A', 'B', 'C'], ['D']) on BenchmarkClasses


学習用とテスト用にデータを分割するのに問題が生じています。@benchmark_name
len(self.train_x) == 3
len(self.train_y) ==  0
len(self.test_x) == 1
len(self.test_y) == 1
MAPEを算出するのに問題が生じました。@sp
test_ratio=0.5
train_list, test_list = (['A', 'B'], ['C', 'D']) on BenchmarkClasses


学習用とテスト用にデータを分割するのに問題が生じています。@benchmark_name
len(self.train_x) == 2
len(self.train_y) ==  0
len(self.test_x) == 2
len(self.test_y) == 1
MAPEを算出するのに問題が生じました。@sp
test_ratio=0.8
train_list, test_list = (['A'], ['B', 'C', 'D']) on BenchmarkClasses


学習用とテスト用にデータを分割するのに問題が生じています。@benchmark_name
len(self.train_x) == 1
len(self.train_y) ==  0
len(self.test_x) == 3
len(self.test_y) == 1
MAPEを算出するのに問題が生じました。@sp


In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

##### ベンチマーククラスをCに固定して各関数で最も予測に適したモデルを算出する処理 #####

# 変数：test_ratios
# テスト用データとする割合のリスト
test_ratios = [0.3, 0.4, 0.5, 0.6, 0.7]

i = 3
test_ratio = test_ratios[i]

# 変数：MapeTable_AtFt
# ベンチマークFTのMAPE表
MapeTable_AtFt = pd.read_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedClassTrain_{test_ratio}.csv")
MapeTable_AtFt = MapeTable_AtFt.set_index("function name")
# 変数：MapeTableColumns_AtFt, MapeTableIndex_AtFt
# ベンチマークFTのMAPE表の列名・行名
MapeTableColumns_AtFt= MapeTable_AtFt.columns.to_numpy()
MapeTableIndex_AtFt = MapeTable_AtFt.index.to_numpy()

# 変数：ModelDataFrame_AtFt
# ベンチマークFTにおける各関数の学習済みモデルが格納されている
if ( benchmarks[i] == "bt" or benchmarks[i] == "sp" ):
    processes = processes_onlyBTSP
else:
    processes = processes_excludeBTSP
FixedClassList_AtFt = return_fixed_class(BenchMark = benchmarks[i], Processes = processes, FixedBenchMarkClass = "C")
FixedClassDataFrame_AtFt = pd.concat(FixedClassList_AtFt , axis=1)
DictData_AtFt = return_dict_Data(FixedClassDataFrame_AtFt)
ModelDataFrameSourceList_AtFt = []
x_list = DictData_AtFt['rowData']
for FunctionName in MapeTableIndex_AtFt:
    y_list = DictData_AtFt[FunctionName]
    ModelDataFrameSourceList_AtFt.append(return_Model_row_list(x=x_list, y=y_list, function_name=FunctionName, test_ratio=test_ratio, train=True))
MapeTable_AtFt
ModelDataFrameSourceListCollumnsName = ["FunctionName", "ModelLin", "ModelLog", "ModelIp", "ModelBranch"]
ModelDataFrame_AtFt = pd.DataFrame(ModelDataFrameSourceList_AtFt)
ModelDataFrame_AtFt.columns = ModelDataFrameSourceListCollumnsName
ModelDataFrame_AtFt = ModelDataFrame_AtFt.set_index("FunctionName")

# 変数：BestModelsDict
# ベンチマークFTの各関数における最適なモデルが格納されている
# 辞書の形式は次の通り
# <関数名>:<自作クラスによる最適モデル>
BestModelsDict = {}
ModelNames = copy.deepcopy(ModelDataFrameSourceListCollumnsName)
ModelNames.remove("FunctionName")
for FunctionName in MapeTableIndex_AtFt:
    MapeInFunction = MapeTable_AtFt.loc[FunctionName].to_list()
    SmallestModelIndex = MapeInFunction.index(min(MapeInFunction))
    SmallestModelName = ModelNames[SmallestModelIndex]
    BestModelsDict[FunctionName] = ModelDataFrame_AtFt.at[FunctionName, SmallestModelName]

BestModelsDict

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

##### 実行プロセス数を256に固定して各関数で最も予測に適したモデルを算出する処理 #####

# 変数：test_ratios
# テスト用データとする割合のリスト
BenchmarkClasses = ["A", "B", "C", "D"]
BenchmarkClasses_on_num = [1, 4, 16, 256]
test_ratios = [0.3, 0.5, 0.8]

i = 3
test_ratio = test_ratios[i]

# 変数：MapeTable_AtFt
# ベンチマークFTのMAPE表
MapeTable_AtFt = pd.read_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedProcessTrain_{test_ratio}.csv")
MapeTable_AtFt = MapeTable_AtFt.set_index("function name")
# 変数：MapeTableColumns_AtFt, MapeTableIndex_AtFt
# ベンチマークFTのMAPE表の列名・行名
MapeTableColumns_AtFt= MapeTable_AtFt.columns.to_numpy()
MapeTableIndex_AtFt = MapeTable_AtFt.index.to_numpy()

# 変数：ModelDataFrame_AtFt
# ベンチマークFTにおける各関数の学習済みモデルが格納されている
if ( benchmarks[i] == "bt" or benchmarks[i] == "sp" ):
    processes = processes_onlyBTSP
else:
    processes = processes_excludeBTSP
FixedClassList_AtFt = return_fixed_class(BenchMark = benchmarks[i], Processes = processes, FixedBenchMarkClass = "C")
FixedClassDataFrame_AtFt = pd.concat(FixedClassList_AtFt , axis=1)
DictData_AtFt = return_dict_Data(FixedClassDataFrame_AtFt)
ModelDataFrameSourceList_AtFt = []
x_list = DictData_AtFt['rowData']
for FunctionName in MapeTableIndex_AtFt:
    y_list = DictData_AtFt[FunctionName]
    ModelDataFrameSourceList_AtFt.append(return_Model_row_list(x=x_list, y=y_list, function_name=FunctionName, test_ratio=test_ratio, train=True))
MapeTable_AtFt
ModelDataFrameSourceListCollumnsName = ["FunctionName", "ModelLin", "ModelLog", "ModelIp", "ModelBranch"]
ModelDataFrame_AtFt = pd.DataFrame(ModelDataFrameSourceList_AtFt)
ModelDataFrame_AtFt.columns = ModelDataFrameSourceListCollumnsName
ModelDataFrame_AtFt = ModelDataFrame_AtFt.set_index("FunctionName")

# 変数：BestModelsDict
# ベンチマークFTの各関数における最適なモデルが格納されている
# 辞書の形式は次の通り
# <関数名>:<自作クラスによる最適モデル>
BestModelsDict = {}
ModelNames = copy.deepcopy(ModelDataFrameSourceListCollumnsName)
ModelNames.remove("FunctionName")
for FunctionName in MapeTableIndex_AtFt:
    MapeInFunction = MapeTable_AtFt.loc[FunctionName].to_list()
    SmallestModelIndex = MapeInFunction.index(min(MapeInFunction))
    SmallestModelName = ModelNames[SmallestModelIndex]
    BestModelsDict[FunctionName] = ModelDataFrame_AtFt.at[FunctionName, SmallestModelName]

BestModelsDict

In [ ]:


# ##### ベンチマーククラスをもとにMAPE表を作成する処理 #####

# for test_ratio in test_ratios:
#     print(f"test_ratio={test_ratio}")
#     print(f"train_list, test_list = {split_by_ratio(BenchmarkClasses, test_ratio)} on BenchmarkClasses")
#     print(f"train_list, test_list = {split_by_ratio(BenchmarkClasses_on_num, test_ratio)} on BenchmarkClasses_on_num")
#     fixed_process_list = [0] * len(benchmarks)
#     for i in range(len(benchmarks)):
#         fixed_process_list[i] = return_fixed_process(BenchMark=benchmarks[i], BenchMarkClasses=BenchmarkClasses, FixedProcess=32)

#     fixed_process_DataFrame = [0] * len(benchmarks)
#     for i in range(len(benchmarks)):
#         fixed_process_DataFrame[i] = pd.concat(fixed_process_list[i], axis=1)

#     # テーブルの元となるDataFrameを作成する
#     MapeTable_source = []
#     for i in range(len(fixed_process_list)):
#         dict_data = return_dict_Data(fixed_process_DataFrame[i])
#         dict_data['rowData'] = BenchmarkClasses_on_num

#         try:
#             MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio)
#         except:
#             print(f"MAPEを算出するのに問題が生じました。@{benchmarks[i]}")
#             continue
        
#         MapeTable_per_benchmark = return_MapeTable_per_benchmark(dict_data, test_ratio=test_ratio)
#         MapeTable_per_benchmark.to_csv(f"./tmp_GenerateResources/{benchmarks[i]}_FixedProcess_{test_ratio}.csv")
#         MapeTable_row = return_MapeTable_row(MapeTable_per_benchmark, benchmarks[i])
#         MapeTable_source.append(MapeTable_row)
#     # テーブルの元となるDataFrameにつける列名のリスト
#     MapeTable_source_column_name = ["線形モデル", "対数モデル", "反比例モデル", "分岐モデル", "ベンチマーク名"]
#     MapeTable = pd.DataFrame(MapeTable_source)
#     MapeTable.columns = MapeTable_source_column_name
#     MapeTable = MapeTable.set_index('ベンチマーク名')
#     # テーブルを欲しい形でCSVとして出力する
#     save_MapeTable(MapeTable, suffix=f"FixedProcess{test_ratio}")

